# Task 4.5: Data Consistency Checks

## Importing Libraries and Data

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Create path to folder
path = r'/Users/caitlin/Documents/Instacart Basket Analysis 12.2021'

In [3]:
# Confirming creation of path
path

'/Users/caitlin/Documents/Instacart Basket Analysis 12.2021'

In [8]:
# Import products data from CSV file
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

In [9]:
# Import orders data from CSV file
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = False)

## Test Dataframe - Mixed-Type Data

In [11]:
# Create a dataframe
df_test = pd.DataFrame()

In [12]:
# Create a mixed type column
df_test['mix'] = ['a', 'b', 1, True]

In [13]:
df_test.head()

mix
0     a
1     b
2     1
3  True

In [15]:
# Check for mixed types
for col in df_test.columns.tolist():
    weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1) 
    if len (df_test[weird]) > 0:
        print (col)

mix


In [17]:
# Change mixed to string
df_test['mix'] = df_test['mix'].astype('str')

## Missing Values

In [19]:
# Find null values
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [22]:
# View null values by creating a subset
df_nan = df_prods[df_prods['product_name'].isnull() == True]

In [21]:
df_nan

product_id product_name  aisle_id  department_id  prices
33             34          NaN       121             14    12.2
68             69          NaN        26              7    11.8
115           116          NaN        93              3    10.8
261           262          NaN       110             13    12.1
525           525          NaN       109             11     1.2
1511         1511          NaN        84             16    14.3
1780         1780          NaN       126             11    12.3
2240         2240          NaN        52              1    14.2
2586         2586          NaN       104             13    12.4
3159         3159          NaN       126             11    13.1
3230         3230          NaN       120             16    14.4
3736         3736          NaN        41              8    14.8
4283         4283          NaN        77              7    14.4
4790         4790          NaN        91             16    14.5
38187       38183          NaN        39             12    20.9
40444       40440          NaN       120             16    14.8

In [24]:
# Determine which strategy to deal with nulls
# First check size of dataset
df_prods.shape

(49693, 5)

In [28]:
# Create new dataframe
# Note: Use of False v. True means original dataframe is in place this is new dataframe
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [26]:
# Check null values have been removed
df_prods_clean.shape

(49677, 5)

## Duplicates

In [29]:
# Find full duplicates
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [30]:
df_dups

product_id                                       product_name  \
462           462                  Fiber 4g Gummy Dietary Supplement   
18459       18458                                         Ranger IPA   
26810       26808               Black House Coffee Roasty Stout Beer   
35309       35306  Gluten Free Organic Peanut Butter & Chocolate ...   
35495       35491                            Adore Forever Body Wash   

       aisle_id  department_id  prices  
462          70             11     4.8  
18459        27              5     9.2  
26810        27              5    13.4  
35309       121             14     6.8  
35495       127             11     9.9

In [31]:
# Drop duplicates
# Determine size before dropping
df_prods_clean.shape

(49677, 5)

In [32]:
# Drop duplicates from dataset
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [34]:
# Check duplicates have been removed
df_prods_clean_no_dups.shape

(49672, 5)

## Export Checked Dataframe

In [35]:
# Export new dataframe with no duplicates and no nulls
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data','Prepared Data','products_checked.csv'))

# Task Questions

## Question 1 

Please see above for consistency checks on df_prods.

## Question 2

In [36]:
# Run describe on products dataframe
df_prods.describe()

product_id      aisle_id  department_id        prices
count  49693.000000  49693.000000   49693.000000  49693.000000
mean   24844.345139     67.770249      11.728433      9.994136
std    14343.717401     38.316774       5.850282    453.519686
min        1.000000      1.000000       1.000000      1.000000
25%    12423.000000     35.000000       7.000000      4.100000
50%    24845.000000     69.000000      13.000000      7.100000
75%    37265.000000    100.000000      17.000000     11.200000
max    49688.000000    134.000000      21.000000  99999.000000

Nothing looks questionable about the department ID column. It is consistent with the names and number of 
departments (21 total) I found in the data dictionary exercise in Task 4.4. For price, 99,999 for a 
grocery store item seems very high, especially considering the mean is about 10. I would investigate
what that item is to see why it is marked so high. To have 134 aisles also seems a little odd, but it 
could be that aisle names are changed (such as calling an aisle "back to school" in August/September).
It is consistent with the mean, so I would take a look, but it may be that there are, in fact, 134 different
aisle IDs.

## Questions 3 and 4

In [39]:
# Check for mixed types in orders dataframe
for col in df_ords.columns.tolist():
    weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1) 
    if len (df_ords[weird]) > 0:
        print (col)

There are no mixed-types in the the orders dataframe.

## Question 5

In [41]:
# Check for missing values in orders dataframe
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

There are over 200,000 missing values in the days_since_prior_order column. This potentially makes sense.
If a customer did not have a previous order, there would not be a value in that column. The fact that there
are so many of them leads me to believe this is the case.

In [42]:
# Examine null values by creating subset for these values
df_nan_ords = df_ords[df_ords['days_since_prior_order'].isnull() == True]

In [43]:
df_nan_ords

Unnamed: 0  order_id  user_id eval_set  order_number  \
0                 0   2539329        1    prior             1   
11               11   2168274        2    prior             1   
26               26   1374495        3    prior             1   
39               39   3343014        4    prior             1   
45               45   2717275        5    prior             1   
...             ...       ...      ...      ...           ...   
3420930     3420930    969311   206205    prior             1   
3420934     3420934   3189322   206206    prior             1   
3421002     3421002   2166133   206207    prior             1   
3421019     3421019   2227043   206208    prior             1   
3421069     3421069   3154581   206209    prior             1   

         orders_day_of_week  order_hour_of_day  days_since_prior_order  
0                         2                  8                     NaN  
11                        2                 11                     NaN  
26                        1                 14                     NaN  
39                        6                 11                     NaN  
45                        3                 12                     NaN  
...                     ...                ...                     ...  
3420930                   4                 12                     NaN  
3420934                   3                 18                     NaN  
3421002                   6                 19                     NaN  
3421019                   1                 15                     NaN  
3421069                   3                 11                     NaN  

[206209 rows x 8 columns]

In looking at the nulls, I now realize that likely the previous order column is null is because each of these customers only ordered once. I will double check.

In [46]:
# Look at values in null column
# Mean
df_nan_ords['order_number'].mean()

1.0

In [47]:
# Value Counts
df_nan_ords['order_number'].value_counts()

1    206209
Name: order_number, dtype: int64

In [49]:
# Check for duplicates in user ID column to confirm each null is a unique customer
df_nan_ords.user_id.duplicated().sum()

0

Even though calculating the mean got me the answer I was looking for, I wanted to learn how to count values, so 
I looked it up online. Both caluculations show the same thing - all 200,000+ of the orders that show null
values in prior order column have only ordered once. To double check, I also checked for duplicates of user IDs
in that subset and confirmed they were all unique customers. 

## Question 6

In [51]:
# Address missing values
# First check size of dataframe
df_ords.shape

(3421083, 8)

The missing values equal about 6 percent of the total number, so deleting them is an option (because it leaves
over 90 percent of the values). If all of the orders where customers have just ordered once are nulls, however,
deleting these rows would eliminate them all together. Since customers all customers who have only ordered once
are likely important to the company, I would not delete them, even though they are a small number.

In [72]:
# Determine how many customers with only one order in orders dataframe
df_ords['order_number'].value_counts()[1]

206209

In [81]:
# Determine if null orders are same number as customers with only one order
df_ords['days_since_prior_order'].isnull().value_counts () 

False    3214874
True      206209
Name: days_since_prior_order, dtype: int64

The number of customers who have only ordered once is equal to the number of nulls in the orders dataframe - not
just the subset. This tells me we need to keep them. I am going to create a new variable to act as a flag - the first option in the exercise. This will require creating a new column, which I will call single_order and then 
indicating if it is true or not. I do not want to overright the existing dataframe, so I will create a new one.

In [83]:
# Create new dataframe
df_ords_clean = df_ords

In [85]:
# Create new column
df_ords_clean['single_order'] = df_ords['days_since_prior_order'].isnull() == True

In [86]:
# Check new dataframe
df_ords_clean.head()

Unnamed: 0  order_id  user_id eval_set  order_number  orders_day_of_week  \
0           0   2539329        1    prior             1                   2   
1           1   2398795        1    prior             2                   3   
2           2    473747        1    prior             3                   3   
3           3   2254736        1    prior             4                   4   
4           4    431534        1    prior             5                   4   

   order_hour_of_day  days_since_prior_order  single_order  
0                  8                     NaN          True  
1                  7                    15.0         False  
2                 12                    21.0         False  
3                  7                    29.0         False  
4                 15                    28.0         False

## Questions 7 and 8

In [87]:
# Find duplicates in orders dataset
df_dups_ords = df_ords_clean[df_ords_clean.duplicated()]

In [88]:
df_dups_ords

Empty DataFrame
Columns: [Unnamed: 0, order_id, user_id, eval_set, order_number, orders_day_of_week, order_hour_of_day, days_since_prior_order, single_order]
Index: []

In [90]:
df_dups_ords.shape

(0, 9)

There do not appear to be any duplicate values in this dataframe.

## Question 9

In [91]:
# Export orders data
df_ords_clean.to_csv(os.path.join(path, '02 Data','Prepared Data','orders_checked.csv'))

The products cleaned data was exporting above, during the task.